In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE OverloadedLists #-}

In [2]:
import Network.HTTP.Req
import qualified Data.ByteString as B
import Data.Default.Class
import Data.Aeson
import Data.Aeson.Text
import Data.Monoid ((<>))
import qualified Data.Text as T

email <- B.init <$> B.readFile "./email"
key   <- B.init <$> B.readFile "./api-key"

emailT <- T.pack . init <$> readFile "./email"

narrow :: Value
narrow = Array
    [ object ["operator" .= String "sender", "operand" .= String emailT]
    , object ["operator" .= String "stream", "operand" .= String "music"]
    ]

params
    =  "anchor"     =: (0  :: Int)
    <> "num_before" =: (0  :: Int)
    <> "num_after"  =: (1  :: Int)
    <> "narrow"     =: (encodeToLazyText narrow)

param = "stream" =: "music"


r = req GET (https "recurse.zulipchat.com" /: "api" /: "v1" /: "messages") NoReqBody jsonResponse ((basicAuth email key) <> params)

r' = req GET
    (https "recurse.zulipchat.com" /: "api" /: "v1" /: "get_stream_id")
    NoReqBody jsonResponse
    ((basicAuth email key) <> param)

res <- responseBody <$> runReq def r :: IO Value
-- res